# Importing libraries
initiating pyspark

In [1]:
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB

In [2]:
import os
import re
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [3]:
# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
# might change later the memory
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("Proteindata spark application")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 20:37:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=Proteindata spark application>

In [6]:
base_path = '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif'
base_path_edit = '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/{}'
file_names = os.listdir(base_path)
file_list = [base_path_edit.format(i) for i in file_names]
files_rdd = sc.parallelize(file_list)

In [7]:
files_rdd.collect()

['/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57867-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57748-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q60271-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q58214-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q58020-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57928-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57607-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q58152-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q59050-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57775-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q58229-F1-model_v4.cif',
 '/mapd-workspace/data_all/UP000000805_2432

hashtag for discriminating different parts , keys are marked with beginning _

In [ ]:
#pdbl = PDBList()
#pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

In [ ]:
schema='aa string, phi double, psi double'

def seq_parsing(file_path):
    file_model = file_path.split(".")[0]
    cif_parser = MMCIFParser()
    #structure = cif_parser.get_structure("3goe", "3goe.cif")
    structure = cif_parser.get_structure(file_model, file_path)
    model0 = structure[0]
    chain_A = model0['A']  # and we get chain A
    # dictionary converting 3-letter codes to 1-letter codes
    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    #print(''.join(sequence))
    return ''.join(sequence)

data_rdd = files_rdd.flatMap(lambda file: seq_parsing(file))
data_rdd.collect()[:10]
    

In [ ]:
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []


for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


#structure.internal_to_atom_coordinates(verbose = True)
#io = PDBIO() #this is to write a pdb file again
#io.set_structure(structure)#set structure, the structure you wan tin the pdb file
#io.save('AF-O53113-F1-model_v4_coded.pdb',  preserve_atom_numbering=True)

#structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)

angles = np.vstack((psi,phi))

np.shape(angles)

In [7]:
def parse_cif(file):
    with open(file, 'r') as fin:
        # load lines 
        lines = fin.readlines()
        print(lines)

    for line in lines:
        key_pattern = "^_.*\w"
        keys = re.findall(key_pattern,line)

    return keys

data_rdd = files_rdd.flatMap(lambda file: parse_cif(file))
data_rdd.collect()[:10]
    

[]

In [16]:
df = spark.read.csv('/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57867-F1-model_v4.cif')
df.show()

+--------------------+
|                 _c0|
+--------------------+
|   data_AF-Q57867-F1|
|                   #|
|_entry.id AF-Q578...|
|                   #|
|               loop_|
|   _atom_type.symbol|
|                  C |
|                  N |
|                  O |
|                  S |
|                   #|
|               loop_|
|  _audit_author.name|
|_audit_author.pdb...|
|"Jumper, John"   ...|
|"Evans, Richard" ...|
|"Pritzel, Alexand...|
|"Green, Tim"     ...|
|"Figurnov, Michae...|
|"Ronneberger, Ola...|
+--------------------+
only showing top 20 rows



In [25]:
with open('/mapd-workspace/data_all/UP000000805_243232_METJA_v4/cif/AF-Q57867-F1-model_v4.cif', 'r') as fin:
    # load lines 
    lines = fin.readlines()
    #print(lines)
keys_list = []
for line in lines:
    key_pattern = "^_.*\w"
    keys = re.findall(key_pattern,line)
    if len(keys) != 0:
        keys_list.append(keys[0].split(' ')[0])
#print(keys_list)

fields = [StructField(field_name, StringType(), True)
      for field_name in keys_list]
schema = StructType(fields)
print(schema)

StructType([StructField('_entry.id', StringType(), True), StructField('_atom_type.symbol', StringType(), True), StructField('_audit_author.name', StringType(), True), StructField('_audit_author.pdbx_ordinal', StringType(), True), StructField('_audit_conform.dict_location', StringType(), True), StructField('_audit_conform.dict_name', StringType(), True), StructField('_audit_conform.dict_version', StringType(), True), StructField('_chem_comp.formula', StringType(), True), StructField('_chem_comp.formula_weight', StringType(), True), StructField('_chem_comp.id', StringType(), True), StructField('_chem_comp.mon_nstd_flag', StringType(), True), StructField('_chem_comp.name', StringType(), True), StructField('_chem_comp.pdbx_synonyms', StringType(), True), StructField('_chem_comp.type', StringType(), True), StructField('_citation.book_publisher', StringType(), True), StructField('_citation.country', StringType(), True), StructField('_citation.id', StringType(), True), StructField('_citation.

In [26]:
lines.split('#')

AttributeError: 'list' object has no attribute 'split'